In [264]:
from AlgoTools import AlgoTools
from Trader import Trader
import pandas as pd
import time
import datetime
import pickle
import numpy as np
import plotly.express as px
import plotly.graph_objects as go


# Processing the data
- The data has already been grouped into days and converted to date time objects
- Objective is to shuffle the days, then create a train-test split as I will be training on indivuals days

In [200]:
with open('SP-groupedByDays.pkl', 'rb') as f:
    spDaily = pickle.load(f)

In [201]:
with open('spydf.pickle', 'rb') as f:
    tickData = pickle.load(f)

In [235]:
import random
random.shuffle(spDaily)

In [203]:
trainLen = int(len(spDaily)*0.75)

In [204]:
trainData=spDaily[:trainLen]
testData=spDaily[trainLen:]

In [252]:
tempData=spDaily[10:11]

# Creating the Algorithim

In [253]:
algoTools = AlgoTools(tickData)

In [298]:
def Algo(data,r, a_L, a_S, E_e, stopProf, stopLoss):
    #data = tempData
   # r, a_L, a_S, E_e, stopProf, stopLoss = params
    trader = Trader()
    #for each day in the data
    for d in range(len(data)):
        #for each time stamp in the day
        index = 1
        
        for t in range(data[d].index[2] , data[d].index[len(data[d])-1]):
            index = index + 1
            
            #close out because market is closing                
            ti = tickData.loc[t,"Time2"].hour*100 + tickData.loc[t,"Time2"].minute
            if (1513<ti<1520 or 2330<ti<2359):
                if trader.getLastTradePos() != 0:
                    trader.closeOut(data[d].iloc[index,:])
                continue
            
            


            
            #if the acceleration is greater than or equal to acceleration constant Long
            elif (algoTools.getAccel(t) >= a_L ) and trader.getLastTradePos() == 0:
                trader.goLong(data[d].iloc[index,:],"A_L reached")
            
            #same as previous statment but for shorting
            #elif  ( algoTools.getAccel(t) <= a_S) and trader.getLastTradePos() == 0:
            #    trader.goShort(data[d].iloc[index,:],"A_S reached")
                
            
            
            
            
           
            
            #-----Closing Conditions------
            
            
            #as per calclus reasoing if the first deravitive is = 0 it has found a local extrema. therfore close out 
            #E_e is error of extrema
          
            elif algoTools.getRate(t) <= E_e and trader.getLastTradePos() != 0:
                trader.closeOut(data[d].iloc[index,:],"E_e reached")
                
#             elif trader.getLastTradePos() == 1 and algoTools.getAccel(t) <a_S:
#                 trader.closeOut(data[d].iloc[index,:],"Acel Switched")
            
#             elif trader.getLastTradePos() == -1 and algoTools.getAccel(t) > a_L:
#                 trader.closeOut(data[d].iloc[index,:],"Acel Switched")
               
            
            #stop profit (in points)
            elif (trader.getLastTradePos() == 1 and data[d].loc[t,"price"] - trader.getLastTradePrice() >= stopProf ) or (trader.getLastTradePos() == -1 and trader.getLastTradePrice() - data[d].loc[t,"price"] >= stopProf):
                trader.closeOut(data[d].iloc[index,:],"Stop Prof reached")
                
            
            #stop loss (in points)
            elif trader.getLastTradePos() == 1 and (trader.getLastTradePrice()-data[d].loc[t,"price"] ) >= stopLoss:
                trader.closeOut(data[d].iloc[index,:],"Stop Loss reached L")
                
                
            elif trader.getLastTradePos() == -1 and (data[d].loc[t,"price"]-trader.getLastTradePrice()) >= stopLoss:
                trader.closeOut(data[d].iloc[index,:],"Stop Loss reached S")
                


            
                
            
            
                
            
    return trader

In [302]:
trader = Algo(tempData,1,0.4,-0.4,0.005,0.5,0.5)

In [303]:
trader.getTradeInfo()

{'Total Return': 20.000000000015916,
 'Max Return': 35.000000000002274,
 'Min Return': -10.000000000002274}

In [304]:
trader.getLR()

0    0.6
dtype: float64

In [305]:
tradeData = trader.getTradeData()
tradeData

,Position,TradePrice,TradeTime,Method
0,1.0,1057.5,2003-11-26 03:50:16,A_L reached
1,0.0,1057.4,2003-11-26 03:50:16,E_e reached
2,1.0,1057.3,2003-11-26 07:30:09,A_L reached
3,0.0,1058.0,2003-11-26 07:30:09,Stop Prof reached
4,1.0,1057.3,2003-11-26 07:32:10,A_L reached
5,0.0,1057.4,2003-11-26 07:32:18,E_e reached
6,1.0,1056.2,2003-11-26 08:47:04,A_L reached
7,0.0,1056.2,2003-11-26 08:47:06,E_e reached
8,1.0,1056.0,2003-11-26 09:00:10,A_L reached
9,0.0,1055.8,2003-11-26 09:00:24,E_e reached


In [306]:
fig = px.line(tempData[0], x='Time2', y='price', title='Price Over Time', labels={'Time': 'Time', 'price': 'Price'})
scatter = go.Scatter(
    x=tradeData['TradeTime'], 
    y=tradeData['TradePrice'], 
    mode='markers', 
    marker=dict(color='red'), 
    name='Trade Points',
    text=tradeData['Method']  # Add labels from the 'Method' column
    #textposition='top center'  # Position of the text labels
)
# Add scatter plot to the line plot
fig.add_trace(scatter)

fig.show()